# Title conditioned GAN training

In [1]:
from transformers import BartTokenizer, BartModel,BartForConditionalGeneration
from torch.distributions import Categorical
import pandas as pd
import torch.nn as nn
from undecorated import undecorated
import numpy as np
from datasets import load_metric
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
from transformers import  Seq2SeqTrainingArguments, Seq2SeqTrainer,DataCollatorForSeq2Seq
metric = load_metric("rouge")
import torch.optim as optim
import torchvision.utils as vutils
import types
import matplotlib.pyplot as plt
from torch.distributions import Categorical
import random


<ipython-input-1-7dd2d3f9d1d8>:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [2]:
from gan_utils import get_end_mask,get_valid_mask,Discriminator_Titles,discriminator_train_title, generate_random_input,reinforce_loss,Generator

In [3]:
from utils import PoemDataset,encode_sentences

In [4]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
#generator = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Data


In [5]:
processed_data_dir =  '/home/alexander/nlg-project/data/'
test_df = pd.read_csv(processed_data_dir + 'test_data.csv')
test_model = encode_sentences(tokenizer,test_df)

train_df = pd.read_csv(processed_data_dir + 'train_data.csv')
train_model = encode_sentences(tokenizer,train_df)

In [17]:
#logprob

In [28]:
#_,logprob = simple_decoding(generator,noise)

In [6]:
train_ds = PoemDataset(train_model)
eval_ds = PoemDataset(test_model)

In [30]:
#discriminator = discriminator.cuda()

# Train

In [31]:
#fixed_noise = generate_random_input(bsize,'cuda')

In [7]:
PATH = '/home/alexander/nlg-project/GAN_models/'

In [11]:

bsize = 12
generator =  Generator().cuda()#BartForConditionalGeneration.from_pretrained('facebook/bart-base').cuda()
#model = TheModelClass(*args, **kwargs)
#generator.load_state_dict(torch.load(PATH + 'generator.pt')['model_state_dict'])
discriminator = Discriminator_Titles().cuda()
#discriminator.load_state_dict(torch.load(PATH + 'discriminator.pt')['model_state_dict'])
criterion = nn.BCEWithLogitsLoss(reduction='none')

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = generate_random_input(bsize,'cuda')


real_label = 1.
ewma_reward = 0.0#torch.tensor(0.0)
fake_label = 0.
lr = .000001
beta1 = .9

optimizerD = optim.Adam(discriminator.parameters(), lr=lr*.05, betas=(beta1, 0.99)) 
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.99))

In [26]:
with open("G_losses_2.txt", "w") as f:
    for s in G_losses:
        f.write(str(s) +"\n")

# Training loop (takes a long long time)

In [12]:
# Training Loop
device = 'cuda'


G_losses = []
D_losses = []
iters = 0
num_epochs = 30

train_dl = DataLoader(train_ds,batch_size=bsize,num_workers=2)
print("Starting Training Loop...")

for epoch in range(num_epochs):
    print("Label Noise:",0.05*.9**epoch)

    for i, data in enumerate(train_dl, 0):
       # print(i)
        
        if len(data['input_ids']) < bsize:
            continue
       # print(data)
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        

        real_inputs = data['input_ids'].to(device)
        real_input_attention = data['attention_mask'].to(device)
        real_input_test = {'input_ids':real_inputs,'attention_mask':real_input_attention}
        
        
        real_gpu = data['label_ids'].to(device)
        attention_gpu = data['decoder_attention_mask'].to(device)
        b_size = real_gpu.size(0)
        #
        label = torch.full((b_size,), 1.0 , dtype=torch.float, device=device) #.95 instead of real to inject noise
        label = torch.bernoulli(label)
       # print(label)
        # Forward pass real batch through D
        real_mask = get_valid_mask(real_gpu)

        output_real,loss_real = discriminator_train_title(discriminator,real_gpu[:,1:],label,real_inputs,criterion,mask=real_mask)

        loss_real.backward()

        # Calculate loss on all-real batch

        D_x = output_real.detach().mean().item()

        # Generate batch of latent vectors
        noise = generate_random_input(bsize,device)

        if random.random() > .5:
            titles = noise['input_ids']
            fake,fake_logprobs = generator(noise)
        else:
            titles = real_input_test['input_ids']
            #sometimes the discirminator learns words in real input, this way some will be based on these same words
            fake,fake_logprobs = generator(real_input_test)
            #real_input_test
  
        
        fake_mask = get_valid_mask(fake)
        end_mask = get_end_mask(fake)
  

        label.fill_(0.0) 
        label = torch.bernoulli(label)
       # print(label)
        # Classify all fake batch with D
        output_fake,loss_fake = discriminator_train_title(discriminator,fake,label,titles,criterion,mask=fake_mask)

        loss_fake.backward()

        D_G_z1 = output_fake.detach().cpu().mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = (torch.mean(loss_real.detach()) + torch.mean(loss_fake.detach())).cpu()

        optimizerD.step()
        

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        
        #generator.zero_grad()

        lossG, cumulative_rewards, ewma_reward = reinforce_loss(output_fake,fake_logprobs,.3,.08,epoch,fake_mask,end_mask,ewma_reward)
        if i % 256 == 0:
            print('Disc LogProbs True')
            print(output_real[0:1])
            print('Disc LogProbs False')
            print(output_fake[0:1])
            print('gen loss')
            print(lossG[0:1])
            print('cum rewards')
            print(cumulative_rewards[0:1])
            print('gen logprobs')
            print(fake_logprobs[0:1])
            print(tokenizer.batch_decode(fake[0:1].cpu()))

        #print(lossG)
        lossG = torch.mean(lossG)

        # Calculate gradients for G
        lossG.backward()
        D_G_z2 = lossG.detach().cpu().mean().item()
        # Update G
        
        optimizerG.step()
        generator.zero_grad()
        discriminator.zero_grad()
        #a = 1/0
        # Output training stats
        if i % 128 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_dl),
                     errD.item(), lossG.item(), D_x, D_G_z1, D_G_z2))
          #  print('ewma_reward:',ewma_reward)

        # Save Losses for plotting later
        G_losses.append(lossG.detach().item())
        D_losses.append(errD.detach().item())
        
        del lossG

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = tokenizer.batch_decode(generator.model.generate(fixed_noise['input_ids'],attention_mask=fixed_noise["attention_mask"],max_length=32,num_beams= 50,early_stopping = True))
                print(fake[0:2])
        if (iters % 1000 == 0 and iters != 0):
            print('saving geneartor')
            torch.save({
            'epoch': 15,
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'loss': 0.3355,
            }, PATH + 'generator.pt')
            torch.save({
            'epoch': 15,
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            'loss': 0.3355,
            }, PATH + 'discriminator.pt')


        iters += 1

Starting Training Loop...
Label Noise: 0.05
Disc LogProbs True
tensor([[0.0357, 0.0705, 0.0716, 0.0788, 0.0167, 0.0682, 0.0883, 0.0807, 0.0600,
         0.0562, 0.0406, 0.0327, 0.0387, 0.0276, 0.0303, 0.0400, 0.0413, 0.0485,
         0.0524, 0.0582, 0.0564]], device='cuda:0', grad_fn=<SliceBackward>)
Disc LogProbs False
tensor([[0.0210, 0.0056, 0.0224, 0.0413, 0.0320, 0.0180, 0.0309, 0.0324, 0.0553,
         0.0000, 0.0000]], device='cuda:0', grad_fn=<SliceBackward>)
gen loss
tensor([[-1.9235e-07, -5.2422e-02,  2.8746e-04,  1.9727e-02,  4.1461e-02,
         -6.6696e-03,  2.7344e-02,  3.0963e-03,  1.4474e-01, -0.0000e+00,
         -0.0000e+00]], device='cuda:0', grad_fn=<SliceBackward>)
cum rewards
tensor([[0.0105, 0.0036, 0.0156, 0.0293, 0.0228, 0.0128, 0.0220, 0.0231, 0.2062,
         0.0000, 0.0000]], device='cuda:0', grad_fn=<SliceBackward>)
gen logprobs
tensor([[-5.3406e-05, -5.0101e+00, -1.9445e-01, -1.2996e+00, -4.7418e+00,
         -5.2734e+00, -3.4352e+00, -3.4237e-01, -6.6185e

KeyboardInterrupt: 

# del optimizerG

In [29]:
torch.save({
            'epoch': 15,
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'loss': 0.3355,
            }, PATH + '4_it_nft_generator.pt')
torch.save({
            'epoch': 15,
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            'loss': 0.3355,
            }, PATH + '4_it_nft_discriminator.pt')

In [ ]:
inputs = tokenizer("cats", return_tensors="pt")

In [ ]:
tokenizer.batch_decode(generator.eval().cpu().generate(inputs['input_ids'],attention_mask=inputs["attention_mask"],max_length=128, use_cache=True,
        decoder_start_token_id = tokenizer.pad_token_id,
        num_beams= 50,
        early_stopping = True))

# Test 

In [ ]:
batch_size = 16 

args = Seq2SeqTrainingArguments(
    "BART-base",
    evaluation_strategy = "epoch",
    #learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
   # weight_decay=0.01,
   # save_total_limit=3,
   # num_train_epochs=1,
   # predict_with_generate=True,
    
    learning_rate=1e-4,
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-6,
    max_grad_norm=1.0,
    num_train_epochs=10,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
   # label_names="labels",
    #fp16=True,
  #  use_auth_token=False
    #push_to_hub=True,
)

In [ ]:
#train_ds = PoemDataset(model_input)
trainer = Seq2SeqTrainer(
    model=BartForConditionalGeneration.from_pretrained('BART-base/checkpoint-25000'),
    args=args,
   # data_collator=data_collator,
    train_dataset=train_ds,
    
    eval_dataset=eval_ds,
    #use_auth_token=False,
    
    tokenizer=tokenizer
)

In [ ]:
trainer.evaluate()

In [ ]:
pwd

In [ ]:
mkdir GAN_models

In [ ]:
torch.save({
            'epoch': 15,
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'loss': 0.3355,
            }, PATH + 'generator.pt')

In [ ]:
torch.save({
            'epoch': 15,
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            'loss': 0.3355,
            }, PATH + 'discriminator.pt')

In [ ]:
pwd

In [ ]:
cd GAN_models